In [17]:
from pulp import *
import pandas as pd

In [18]:
# Cargar datos
df_vehiculos = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')
df_clientes = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_distancias = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')

df_distancias.index = df_distancias.columns


In [19]:
# Preparación datos para PuLP
capacidades = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["capacidad_kg"]))
costos = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["costo_km"]))
autonomias = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["autonomia_km"]))

pedidos = dict(zip(df_clientes["cliente"], df_clientes["order_demand"]))
pedidos["Almacén"] = 0  # Aseguramos que el almacén no tenga demanda

distancias = df_distancias.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_clientes["cliente"].tolist() + ["Almacén"]  # Incluir el almacén como un cliente especial
vehiculos = df_vehiculos["vehiculo_id"].tolist()

In [20]:
# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)

# Función objetivo: minimizar costo total
problema += lpSum(
    x[i, j, k] * costos[k] * distancias.get((i, j), 0)
    for i in clientes for j in clientes for k in vehiculos
)

# Restricciones
# Cada cliente debe ser atendido exactamente una vez
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos) == 1

# Respetar la capacidad de los vehículos
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Autonomía de los vehículos, considerando el viaje de ida y vuelta al almacén
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes) \
                + lpSum(x[i, 'Almacén', k] * distancias.get((i, 'Almacén'), 0) for i in clientes if i != 'Almacén') \
                + lpSum(x['Almacén', j, k] * distancias.get(('Almacén', j), 0) for j in clientes if j != 'Almacén') \
                <= autonomias[k]

# Flujo de vehículos: entrada = salida, incluido el almacén
for i in clientes:
    for k in vehiculos:
        if i != 'Almacén':
            problema += lpSum(x[i, j, k] for j in clientes) == lpSum(x[j, i, k] for j in clientes)
        else:
            # Para el almacén, la suma de las salidas debe ser igual a la suma de las entradas
            problema += lpSum(x[i, j, k] for j in clientes if j != i) == lpSum(x[j, i, k] for j in clientes if j != i)

# Restricción de Inicio desde el Almacén
for k in vehiculos:
    problema += lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén') == 1

# Restricción de Retorno al Almacén
for k in vehiculos:
    problema += lpSum(x[i, 'Almacén', k] for i in clientes if i != 'Almacén') == 1

# Asegurar que no haya bucles directos entre clientes sin pasar por el almacén
for i in clientes:
    if i != 'Almacén':
        for j in clientes:
            if j != 'Almacén' and i != j:
                for k in vehiculos:
                    problema += x[i, j, k] + x[j, i, k] <= 1  # No puede haber un viaje de ida y vuelta entre dos clientes

# Restricción adicional para evitar bucles
for k in vehiculos:
    for i in clientes:
        if i != 'Almacén':
            problema += lpSum(x[j, i, k] for j in clientes) - lpSum(x[i, j, k] for j in clientes if j != i) == 0

# Asegurar que q[i, k] solo tenga valor si el vehículo k va al cliente i
for i in clientes:
    for k in vehiculos:
        problema += q[i, k] <= lpSum(x[i, j, k] for j in clientes) * pedidos[i]

# Resolver el problema con un tiempo límite
solver = PULP_CBC_CMD(msg=1, timeLimit=10)  # 300 segundos de límite de tiempo
problema.solve(solver)
print(LpStatus[problema.status])

# Debug: Imprimir decisiones de rutas y cargas
for k in vehiculos:
    for i in clientes:
        for j in clientes:
            if x[i, j, k].value() == 1:
                print(f"Vehículo {k} va de {i} a {j}")


# Verificación de Demanda vs Capacidad Total
total_demand = sum(pedidos[i] for i in clientes if i != 'Almacén')
total_capacity = sum(capacidades[k] for k in vehiculos)
print(f"Total demanda: {total_demand}, Total capacidad: {total_capacity}")

Optimal
Vehículo 1 va de Cliente_11 a Almacén
Vehículo 1 va de Almacén a Cliente_11
Vehículo 2 va de Cliente_3 a Cliente_20
Vehículo 2 va de Cliente_17 a Almacén
Vehículo 2 va de Cliente_19 a Cliente_3
Vehículo 2 va de Cliente_20 a Cliente_19
Vehículo 2 va de Almacén a Cliente_17
Vehículo 3 va de Cliente_2 a Cliente_18
Vehículo 3 va de Cliente_6 a Almacén
Vehículo 3 va de Cliente_13 a Cliente_2
Vehículo 3 va de Cliente_18 a Cliente_13
Vehículo 3 va de Almacén a Cliente_6
Vehículo 4 va de Cliente_1 a Almacén
Vehículo 4 va de Cliente_4 a Cliente_1
Vehículo 4 va de Almacén a Cliente_4
Vehículo 5 va de Cliente_5 a Cliente_7
Vehículo 5 va de Cliente_7 a Cliente_14
Vehículo 5 va de Cliente_8 a Cliente_5
Vehículo 5 va de Cliente_9 a Almacén
Vehículo 5 va de Cliente_14 a Cliente_15
Vehículo 5 va de Cliente_15 a Cliente_8
Vehículo 5 va de Almacén a Cliente_9
Vehículo 6 va de Cliente_10 a Cliente_16
Vehículo 6 va de Cliente_12 a Cliente_10
Vehículo 6 va de Cliente_16 a Almacén
Vehículo 6 va de A

# Resultados corregidos
for k in vehiculos:
    print(f"Vehículo {k}:")
    ruta = []
    actual = 'Almacén'
    visitados = set()  # Para evitar bucles
    while True:
        if actual in visitados and actual != 'Almacén':  # Si ya visitamos este nodo y no es el almacén, algo está mal
            print(f"Error: Bucle detectado para el vehículo {k}.")
            break
        visitados.add(actual)
        
        siguiente = next((j for j in clientes if j != actual and x[actual, j, k].value() == 1), None)
        if siguiente is None:  # Si no hay más movimientos, el vehículo ha vuelto al almacén
            ruta.append(actual)
            ruta.append('Almacén')
            break
        ruta.append(actual)
        actual = siguiente

    if len(ruta) > 1:  # Si la ruta tiene más de un nodo (incluyendo inicio y fin en almacén)
        print(f"  Ruta: {' -> '.join(ruta)}")
        carga_total = sum(pedidos[i] for i in ruta if i != 'Almacén')
        print(f"  Carga total: {carga_total} kg")
        total_distance = sum(x[i, j, k].value() * distancias.get((i, j), 0) for i in clientes for j in clientes if x[i, j, k].value() == 1)
        print(f"  Distancia total: {total_distance} km")
        print(f"  Autonomía: {autonomias[k]} km")
        print(f"  Capacidad: {capacidades[k]} kg")
    else:
        print("  Ruta: No se ha asignado una ruta a este vehículo")
    print()

In [21]:
for k in vehiculos:
    print(f"Vehículo {k}:")
    ruta = []
    actual = 'Almacén'
    visitados = set()  # Para evitar bucles infinitos
    while True:
        # Añadir el nodo actual a la ruta
        ruta.append(actual)
        visitados.add(actual)

        # Buscar el siguiente nodo en la ruta
        siguiente = next((j for j in clientes + ['Almacén'] if j != actual and x[actual, j, k].value() == 1), None)

        if siguiente is None:  # Si no hay más movimientos, la ruta ha terminado
            break

        # Si el siguiente nodo es el almacén, pero quedan clientes sin visitar, continuar la búsqueda
        if siguiente == 'Almacén' and not visitados.issuperset(clientes):
            ruta.append(siguiente)
            actual = siguiente
            continue

        # Si volvemos al almacén y todos los clientes han sido visitados, la ruta termina
        if siguiente == 'Almacén' and visitados.issuperset(clientes):
            ruta.append(siguiente)
            break

        # Avanzar al siguiente nodo
        actual = siguiente

    # Imprimir la ruta final del vehículo
    if len(ruta) > 1:  # Si la ruta tiene más de un nodo
        print(f"  Ruta: {' -> '.join(ruta)}")
        carga_total = sum(pedidos[i] for i in ruta if i != 'Almacén')
        print(f"  Carga total: {carga_total} kg")
        total_distance = sum(
            x[i, j, k].value() * distancias.get((i, j), 0) 
            for i in clientes + ['Almacén'] 
            for j in clientes + ['Almacén'] 
            if x[i, j, k].value() == 1
        )
        print(f"  Distancia total: {total_distance} km")
        print(f"  Autonomía: {autonomias[k]} km")
        print(f"  Capacidad: {capacidades[k]} kg")
    else:
        print("  Ruta: No se ha asignado una ruta a este vehículo")
    print()

Vehículo 1:


KeyboardInterrupt: 